In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("bucketing").master("local[*]").getOrCreate()

In [2]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [3]:
orders = spark.read.option("header",True).option("inferSchema",True).csv("orders.csv")
orders.show(5,False)

+--------+----------+-----------+--------+-------------------+------------+
|order_id|product_id|customer_id|quantity|order_date         |total_amount|
+--------+----------+-----------+--------+-------------------+------------+
|1       |80        |10         |4       |2023-03-20 00:00:00|1003        |
|2       |69        |30         |3       |2023-12-11 00:00:00|780         |
|3       |61        |20         |4       |2023-04-26 00:00:00|1218        |
|4       |62        |44         |3       |2023-08-26 00:00:00|2022        |
|5       |78        |46         |4       |2023-08-05 00:00:00|1291        |
+--------+----------+-----------+--------+-------------------+------------+
only showing top 5 rows



In [4]:
products = spark.read.option("header",True).option("inferSchema",True).csv("products.csv")
products.show(5,False)

+----------+------------+-----------+-------+-----+-----+
|product_id|product_name|category   |brand  |price|stock|
+----------+------------+-----------+-------+-----+-----+
|1         |Product_1   |Electronics|Brand_4|26   |505  |
|2         |Product_2   |Apparel    |Brand_4|489  |15   |
|3         |Product_3   |Apparel    |Brand_4|102  |370  |
|4         |Product_4   |Groceries  |Brand_1|47   |433  |
|5         |Product_5   |Groceries  |Brand_3|244  |902  |
+----------+------------+-----------+-------+-----+-----+
only showing top 5 rows



In [7]:
# Join without bucketing
products_orders = orders.join(products,orders.product_id == products.product_id,"inner")
products_orders.explain()

# we can see in the plan that shuffle sort merge is taking place

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [product_id#18], [product_id#78], Inner
   :- Sort [product_id#18 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(product_id#18, 200), ENSURE_REQUIREMENTS, [id=#84]
   :     +- Filter isnotnull(product_id#18)
   :        +- FileScan csv [order_id#17,product_id#18,customer_id#19,quantity#20,order_date#21,total_amount#22] Batched: false, DataFilters: [isnotnull(product_id#18)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jupyter/data/orders.csv], PartitionFilters: [], PushedFilters: [IsNotNull(product_id)], ReadSchema: struct<order_id:int,product_id:int,customer_id:int,quantity:int,order_date:timestamp,total_amount...
   +- Sort [product_id#78 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(product_id#78, 200), ENSURE_REQUIREMENTS, [id=#85]
         +- Filter isnotnull(product_id#78)
            +- FileScan csv [product_id#78,product_name#79,category#80,brand#81,price#82,

# Bucketing
Now lets create buckets on both the data and store it as a table

In [10]:
orders.write.bucketBy(4, col = "product_id").mode("overwrite").saveAsTable("orders_bucketed")
# After this we can see that there is a folder created called orders_bucketed with 4 files inside it

In [12]:
products.write.bucketBy(4, col = "product_id").mode("overwrite").saveAsTable("products_bucketed")
# Similarly another folder is created for products_bucketed with 4 files inside it

In [13]:
# Now lets read this saved data as table in spark
# As we did saveAsTable, the metadata of this will be stored in spark catalog, so when we can directly read it as tables in spark
# If we did not saveAsTable, then this can only be read as dataframe and not tables

df_orders_bucketed = spark.table("orders_bucketed")
df_products_bucketed = spark.table("products_bucketed")

In [16]:
df_products_orders_joined =  df_orders_bucketed.join(df_products_bucketed, on = "product_id", how="inner")
df_products_orders_joined.explain()
# we can see that there is sortMerge join taking place, but without shuffle being done

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [product_id#231, order_id#230, customer_id#232, quantity#233, order_date#234, total_amount#235, product_name#243, category#244, brand#245, price#246, stock#247]
   +- SortMergeJoin [product_id#231], [product_id#242], Inner
      :- Sort [product_id#231 ASC NULLS FIRST], false, 0
      :  +- Filter isnotnull(product_id#231)
      :     +- FileScan parquet default.orders_bucketed[order_id#230,product_id#231,customer_id#232,quantity#233,order_date#234,total_amount#235] Batched: true, Bucketed: true, DataFilters: [isnotnull(product_id#231)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/jupyter/data/spark-warehouse/orders_bucketed], PartitionFilters: [], PushedFilters: [IsNotNull(product_id)], ReadSchema: struct<order_id:int,product_id:int,customer_id:int,quantity:int,order_date:timestamp,total_amount..., SelectedBucketsCount: 4 out of 4
      +- Sort [product_id#242 ASC NULLS FIRST], false, 0
         +-

# Bucketing In Aggregations

In [17]:
# Groupby without bucketing
df_product_sales = (
    orders
    .groupBy("product_id")
    .agg(F.sum("total_amount").alias("sales"))
)

df_product_sales.explain()
# we can see shuffle

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[product_id#18], functions=[sum(total_amount#22)])
   +- Exchange hashpartitioning(product_id#18, 200), ENSURE_REQUIREMENTS, [id=#158]
      +- HashAggregate(keys=[product_id#18], functions=[partial_sum(total_amount#22)])
         +- FileScan csv [product_id#18,total_amount#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jupyter/data/orders.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<product_id:int,total_amount:int>




In [18]:
# Groupby without bucketing
df_product_sales = (
    df_orders_bucketed
    .groupBy("product_id")
    .agg(F.sum("total_amount").alias("sales"))
)

df_product_sales.explain()
# There is no shuffle

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[product_id#231], functions=[sum(total_amount#235)])
   +- HashAggregate(keys=[product_id#231], functions=[partial_sum(total_amount#235)])
      +- FileScan parquet default.orders_bucketed[product_id#231,total_amount#235] Batched: true, Bucketed: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/jupyter/data/spark-warehouse/orders_bucketed], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<product_id:int,total_amount:int>, SelectedBucketsCount: 4 out of 4




# Bucket pruning (Filters)
Bucketing is also helpful when we filer the data, if we filter the data based on id 100, it will perform hash on 100 and whatever the output is perform a mod (no_of_buckets) and based on output it will directly just read that bucket instead of full scan.

In [20]:
df_product_sales_bucket_pruning = (
    df_orders_bucketed
    .filter(F.col("product_id") == 1)
    .groupBy("product_id")
    .agg(F.sum("total_amount").alias("sales"))
)
df_product_sales_bucket_pruning.explain()
# We can see that SelectedBucketsCount: 1 out of 4 in the last line of plan which tells us that only one bucket is searched for id =1

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[product_id#231], functions=[sum(total_amount#235)])
   +- HashAggregate(keys=[product_id#231], functions=[partial_sum(total_amount#235)])
      +- Filter (isnotnull(product_id#231) AND (product_id#231 = 1))
         +- FileScan parquet default.orders_bucketed[product_id#231,total_amount#235] Batched: true, Bucketed: true, DataFilters: [isnotnull(product_id#231), (product_id#231 = 1)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/jupyter/data/spark-warehouse/orders_bucketed], PartitionFilters: [], PushedFilters: [IsNotNull(product_id), EqualTo(product_id,1)], ReadSchema: struct<product_id:int,total_amount:int>, SelectedBucketsCount: 1 out of 4




In [21]:
spark.stop()